In [0]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm

In [0]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.set_index("Like Count", inplace=True)
data.sort_values("Like Count", inplace=True,ascending=False)
# print(data.head())
data['class'] = ls
index = data["Like Count"].tolist()
data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
data.head()

,id,Page,Date,Time,Like Count,Comment Count,Share Count,Category,Crime_flag,Feed_Img,Headline,Summary,Publish Timestamp,Post Message,Post Description,Post Link,Post Type,Post Img,Status Type,class
23761,23762,Channel NewsAsia,10-07-2018 19:51:37,41,51131,2712,26132,Uncategorized,-,-,-,-,-,THEY'RE OUT! After more than 2 weeks trapped i...,-,https://www.facebook.com/ChannelNewsAsia/photo...,photo,0,added_photos,1
21386,21387,Channel NewsAsia,10-05-2018 21:49:19,45,21377,9418,13938,Main,0,0,Mahathir sworn in as prime minister of Malaysia,"HAPPENING NOW: Dr. Mahathir bin Mohamad, 92, i...",10-05-2018 21:49:19,"HAPPENING NOW: Dr. Mahathir bin Mohamad, 92, i...",-,https://www.facebook.com/ChannelNewsAsia/video...,video,0,added_video,1
23390,23391,Channel NewsAsia,03-07-2018 08:48:32,19,16598,658,8418,Uncategorized,-,-,-,-,-,SAYONARA: This is the Japan team's dressing ro...,-,https://www.facebook.com/ChannelNewsAsia/photo...,photo,0,added_photos,1
20924,20925,Channel NewsAsia,27-04-2018 17:36:38,36,15913,893,8128,Main,0,0,North and South Korea sign joint declaration o...,"WATCH: Smiles, handshakes and a warm hug as th...",27-04-2018 17:36:38,"WATCH: Smiles, handshakes and a warm hug as th...",-,https://www.facebook.com/ChannelNewsAsia/video...,video,0,added_video,1
31626,31627,TODAYonline,25-04-2018 18:08:51,37,14077,2996,33913,Main,0,0,-,LAST GOODBYE: The Singapore Zoo said goodbye t...,25-04-2018 18:08:51,LAST GOODBYE: The Singapore Zoo said goodbye t...,-,https://www.facebook.com/todayonline/photos/a....,photo,0,added_photos,1


In [0]:
model_summary = FastText.load("model_summary.bin")

In [0]:
category = set(data["Category"].values.tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Category"][h]]))
    head_vec.append(sum1)

/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
train = head_vec[:4261]+head_vec[4761:4761*2]
res_train = data["class"][:4261].tolist()+data["class"][4761:4761*2].tolist()
test = head_vec[4261:4761]+head_vec[int(0.8*47611):]
res_test = data["class"][4261:4761].tolist()+data["class"][int(0.8*47611):].tolist()
print(len(train), len(res_train))

9022 9022


In [0]:
model_svm_5050 = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y_50 = svmutil.svm_predict(res_test, test, model_svm_5050)

Accuracy = 76.4641% (7664/10023) (classification)


In [0]:
y_1 = svmutil.svm_predict(res_test[:500], test[:500], model_svm_5050)

Accuracy = 43.6% (218/500) (classification)


In [0]:
y_1[2]

[[-0.98965062949746],
 [0.9987375931434322],
 [0.994271097862571],
 [-1.0016638159183202],
 [-0.9912854550991307],
 [0.9999410473674363],
 [0.9968671131128278],
 [-0.9840023376368573],
 [-0.9977500841280088],
 [0.9985364092676122],
 [-0.98965062949746],
 [0.9972808314998444],
 [0.9984780291306482],
 [0.9962768524294007],
 [0.9956583815534298],
 [1.0023176522382653],
 [-1.0029074664560107],
 [1.000709095482096],
 [0.9993736836855439],
 [-0.9998002987287612],
 [0.9961158880336377],
 [-0.9818543559429312],
 [-0.9918572284736996],
 [-0.9933027756420209],
 [0.998262673794721],
 [0.9978819165325788],
 [0.99099731934642],
 [-0.9967740794469141],
 [-0.9989205606694168],
 [0.9948480846168667],
 [-0.9838436134409555],
 [-0.9863563650257904],
 [-0.98965062949746],
 [0.9998494946106714],
 [-0.9995571623605511],
 [0.9958013010914547],
 [0.9998668730031287],
 [-1.0056702853618673],
 [-0.9963894245037285],
 [0.9958666284005498],
 [0.9938424181994331],
 [0.997687929667164],
 [-0.993612779097082],
 [-1

In [0]:
from sklearn.metrics import confusion_matrix, auc, roc_curve

In [0]:
confusion_matrix(res_test, y_50[0], labels=[0,1])

array([[7446, 2077],
       [ 282,  218]])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y_50[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

0.436

In [0]:
precision = tp/(tp+fp)
precision

0.09498910675381264

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

0.1559928443649374

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y_50[0])
AUC = auc(fpr, tpr)
AUC

0.6089482305996009

In [0]:
svmutil.svm_save_model("model_svm.bin",model_svm_5050)

In [0]:
svmutil.svm_train(res_train, train,'-v 10 -t 2 -c 2')

Cross Validation Accuracy = 61.7601%


61.760141875415655